# 1 **What's new**

- running a simple simulation with the CNS solver
- the important input data to solve the Euler equations
- generating a simple mesh by using the \BoSSS{} internal mesher
- the post-processing of the data, i.e. visualization in Paraview or VisIt

# 1.1 Prerequisites

- the ***BoSSS** framework
- a visualization tool, e.g ***Paraview*** or ***VisIt***
- the knowledge of how to setup a database for ***BoSSS***




# 2 Problem statement
We only briefly describe the non dimensional Euler equations in 2D
>$$\frac{\partial}{\partial t}\vec{U}+\frac{\partial}{\partial x_j}\vec{F}^c_j(U)= 0,$$
where $\vec{U}$ are the conserved flow variables and $\vec{F}^c_x$  are the convective fluxes, i.e.
>$$\vec{U} = 
\begin{pmatrix}  
  \rho \\
  \rho u_i \\
  \rho E
\end{pmatrix} , \quad \textrm{and} \quad \vec{F}^c_i = \begin{pmatrix}
  \rho u_i\\
  \rho u_i u_j + \frac{1}{\gamma \textit{Mach}^2}p \delta_{ij}\\
  u_j (\rho E+p)
\end{pmatrix}.$$



# Note:

In our non dimensional equations, we introduce $\textit{Mach}$. 

These Quick Start tutorials are aimed to show some of the main features of the compressible flow 
solver (**Compressible Navier-Stokes (CNS)**) in the **BoSSS** framework.

Note, that **BoSSS** uses a C\# code based input data and interprets these in 
the ***REPL*** fashion. 

This gives us more flexibility in the way how we can start a 
simulation. 

We can have the traditional way of defining an input file, where we define all 
important parameters in C\# code, or we can also use some predefined functions in the framework 
to generate our input data and manipulate them interactively in the **BoSSSpad**.

As an example, we will simulate the well known isentropic vortex for the Euler equations. 



# 3 Isentropic vortex

ATTENTION: Before you start you need to modify the first line to point to your local **BoSSSPad.dll** :file_folder:

```csharp
#r "BoSSSpad.dll"
```
such that the BoSSSpad.dll can be found (e.g it should look like the following)

```csharp
#r "..\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
```

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


We start with creating a database and loading the namespace.

In [ ]:
/// We create, resp. open a \BoSSS database:
var myDatabase = CreateTempDatabase();

In [ ]:
using CNS;

In [ ]:
typeof(CNS.Program).Assembly.Location

We load the input data from some function within the Framework. 
  
The class **ControlExamples** provides some predefined control sets for 
different typical test cases, i.a. the isentropic vortex. 

For the isentropic vortex, you have to specify 

- path to a database (here: EMPTY path)
- number of cells in each direction (here: 20)
- DG order (here: 2)
- advection velocity of the vortex (here: 1.0)


In [ ]:

int noOfCellsPerDirection = 20;       
var c = ControlExamples_Subsonic.IsentropicVortex(myDatabase.Path,      
                                         noOfCellsPerDirection,2,1.0);
c.savetodb = true;       


Now, we have a set of input data stored in our variable **c** and we 
can have a look at different parameters directly via \code{BoSSSpad}.

# 3.1 How to define/change input data

In this section we will walk you through the most important parameters for 
running the test case.

We start with the grid and use the ability of **BoSSS**
to generate simple grids with its own mesh generator. 

Once we start the simulation, the control object **c** is parsed and the grid defined by the **GridFunc**
is generated on the fly and stored in the database. Here, we defined a 
**delegate** function which returns a uniform cartesian grid.

First, we have to define a 1-D array, which spans from -10 to 10 and is divided into the number 
of cells, which we previously set to **int noOfCellsPerDirection = 20**. 

The function **Grid2D.Cartesian2DGrid** generates a
uniform 2-D grid out of this array by using it for x and y direction
Additionally we specify periodic boundary conditions in x and y direction
by setting 
>**periodicX: true** and **periodicY: true**. 

In [ ]:

c.GridFunc = delegate {       
   double[] nodes = GenericBlas.Linspace(-10, 10, noOfCellsPerDirection + 1);       
   var grid = Grid2D.Cartesian2DGrid(nodes, nodes,       
                                     periodicX: true, periodicY: true);       
   return grid;       
};

Error: (1,1): error CS0103: Der Name "c" ist im aktuellen Kontext nicht vorhanden.

The **CNS solver** is able to solve the Euler and the compressible Navier-Stokes equations. By setting

In [ ]:
c.ActiveOperators

we only use the convective fluxes, i.e the Euler equations, and set it to

In [ ]:
c.ConvectiveFluxType

# Note: 

Optimized means in this case, that this is the classical HLLC flux, but implemented in an optimized version.

As initial conditions, we choose the analytical solution, which can be found in various publications, e.g. Hu (2006).

The Mach number is set in the following:

In [ ]:
c.MachNumber

Further, we have to define a **simulation time**, i.e

In [ ]:
c.Endtime

Finally, we need a **time stepping scheme**

In [ ]:
c.ExplicitScheme

...of **order**...

In [ ]:
c.ExplicitOrder

...to run the simulation.

These are all predefined input values, which were set by calling **ControlExamples_Subsonic.IsentropicVortex(...)**.

Since we are in the interactive mode, we can change them directly.

For example, we can reduce the order of our **timestepping scheme** to 3, 

because we only use DG order 2:

In [ ]:

c.ExplicitOrder = 3;      
c.ExplicitOrder

Or we can change the **Mach number** to

In [ ]:
c.MachNumber = 0.7;    
c.MachNumber

# 3.2 Run a simulation
We adjusted our input values and now we can run a simulation.

In the interactive mode, we can simply execute **\Run()** on the control
object, which will execute the solver:

In [ ]:

c.PrintInterval = 5;
var SI          = c.Run();

The **Run()** command finally returns a session info, which carries some
basic information on the solver run (mainly where ist was stored).

In [ ]:
SI

# 3.3 Console mode
We can also run this simulation in the *traditional* way, which most
of you are familiar with from other academical codes. 

We define an input file, which is nothing else than the above C\# code. 

We can run it by calling
>**CNS.exe -c IsentropicVortex.cs**. 

You can find the input file in 
*ControlExamples folder* in the doc directory.

# 3.4 Postprocessing

We saved our data in the database and lastly we want to postprocess it, 
i.e visualize the individual fields like density, momentum or pressure.

# Note: 

If you have run the simulation in the console mode, you now have to start 
the **BoSSSpad**.

In our example, we find the corresponding session in our first database as first session

In [ ]:
//myDatabase.Sessions.First();

To convert data to the Tecplot format, we just need to export it (in order to run the command delete the comment)

In [ ]:

//myDatabase.Sessions.First().Export().Do()

We can open the folder directly by using (works only in **BoSSSPad**)

In [ ]:
//myDatabase.Sessions.First().Export().Do()

... where we find the files with our data.

For more information about our databases and useful commands for postprocessing,
we refer to our tutorials about the database and the database command overview.